In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
import mlflow

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen

# misc
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read the wide data 
all_wide_data = pd.read_csv('df_final_date_wide_2007.csv')
all_wide_data.shape

(65363, 40)

### Utility Functions

In [36]:
# Function to add the column to the main data
def add_column_to_main_data(data, cluster_data, col_name):
    
    cluster_data['ags5'] = cluster_data['ags5'].apply(fix_ags5)
    
    # Get the cluster and ags 5 and set ags 5 as index 
    cluster_info = cluster_data.set_index('ags5').to_dict()[col_name]
    
    data['cluster'] = '0'
    data['cluster'] = data['ags5'].map(cluster_info)

    # check if the cluster have been allotted correctly 
    print("original cluster data")
    print(cluster_data[col_name].value_counts())
    print("New data")
    print(data.drop_duplicates(subset=['ags5'])['cluster'].value_counts())
    
    return data

In [3]:
# Fix ags 5
def fix_ags5(x):
    """Function to fix the format of ag5 """
    if len(str(x))==4:
        return '0'+str(x)
    else:
        return str(x)
all_wide_data['ags5'] = all_wide_data['ags5'].apply(fix_ags5)

In [4]:
all_wide_data['ags5']

0        01001
1        01001
2        01001
3        01001
4        01001
         ...  
65358    16077
65359    16077
65360    16077
65361    16077
65362    16077
Name: ags5, Length: 65363, dtype: object

## Create ML Flow Evaluation function

In [5]:
def evaluate_var_model(all_data, params):
    
    # Create a runname
    run_name = 'var_' + str(len(params['relevant_cols'])) + 'cols_' + str(params['lag_value']) + 'lag_diff_' + str(params['second_diff'])
    
    ''' Utility Functions '''
    
    # Invert the transformation to get the real forecast
    def invert_transformation(df_train, df_forecast, second_diff=False):
        
        """Revert back the differencing to get the forecast to original scale."""
        df_fc = df_forecast.copy()
        columns = df_train.columns
        for col in columns:        
            # Roll back 2nd Diff
            if second_diff:
                df_fc[str(col)+'_diff'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
            # Roll back 1st Diff
            df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
        return df_fc
    
    # Start the MLflow experiment 
    with mlflow.start_run(run_name=run_name):
        
        # Get list of kreis 
        kreis_list = list(all_data['ags5'].unique())
        
        # Define total metrics 
        total_mse, total_rmse, total_mape = 0, 0, 0
        
        print("Fitting all models", end='')
        # Iterate through each kreis
        for kreis in kreis_list:
            
            print('.', end='')
            
            # Filter data by kreis
            kreis_data = all_wide_data[all_wide_data['ags5'] == kreis]
            
            # Convert date format
            kreis_data['date'] = pd.to_datetime(kreis_data['date'])
            
            # Clip the data before 2010 and after 2019 
            start_date = '2010-01-01'
            end_date = '2020-01-01'
            mask = (kreis_data['date'] >= start_date) & (kreis_data['date'] < end_date)
            useful_data = kreis_data[(mask)]
            
            # Set the date as the index
            useful_data.set_index('date', inplace=True)
            
            # Select the relevant columns from the data
            relevant_cols = params['relevant_cols']
            useful_data = useful_data[relevant_cols]
            
            ''' Modelling Fitting'''
            
            # Split the training and testing data 
            n_obs = params['num_predictions']
            df_train, df_test = useful_data[0:-n_obs], useful_data[-n_obs:]
            
            # Differentiate the data twice if asked else once
            if_second_diff = params['second_diff']
        
            df_differenced = df_train.diff().dropna()
            if if_second_diff:
                df_differenced = df_differenced.diff().dropna()
            
            # Get the lag value 
            lag_value = params['lag_value']
            
            # Feed the model data
            model = VAR(df_differenced)
            
            # Fit the model to lag
            model_fitted = model.fit(lag_value)
            
            '''Forecasting section'''
            # Get the lag order
            lag_order = model_fitted.k_ar

            # Input data for forecasting
            forecast_input = df_differenced.values[-lag_order:]
            
            # Get the forecasts
            fc = model_fitted.forecast(y=forecast_input, steps=n_obs)
            df_forecast = pd.DataFrame(fc, index=useful_data.index[-n_obs:], columns=useful_data.columns + '_diff')
            
            # Invert the forecast results
            df_results = invert_transformation(df_train, df_forecast, second_diff=if_second_diff)
            final_results = df_results.filter(regex='_forecast')
            
            ''' Evaluation '''
            forecast = df_results['unemployment_rate_forecast'].values
            actual = df_test['unemployment_rate']
            mse = mean_squared_error(actual, forecast, squared=True)
            rmse = mean_squared_error(forecast, actual, squared=False)
            mape = np.mean(np.abs(forecast - actual)/np.abs(actual))
            
            # add to the totals
            total_mse += mse
            total_rmse += rmse
            total_mape += mape
            
        ''' Log metrics '''
        
        # Calculate errors
        average_mse = total_mse/len(kreis_list)
        average_rmse = total_rmse/len(kreis_list)
        average_mape = total_mape/len(kreis_list)
        
        # Log model 
#         mlflow.statsmodels.log_model(model, "var_model")
        
        # log params
        mlflow.log_params(params)
        
        # Log errors
        mlflow.log_metric("mse", average_mse)
        mlflow.log_metric("rmse", average_rmse)
        mlflow.log_metric("mape", average_mape)

            

In [6]:
relevant_cols = ['number_of_company_deletions', 'number_of_company_liquidations',
       'number_of_start_ups', 'unemployment_rate', 'registerd_jobs',
       'unemployed', 'employees_social_security_at_work',
       'employees_social_security_at_residence']

In [7]:
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': False,
}

evaluate_var_model(all_wide_data, params)

Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................

In [14]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
#                  'registerd_jobs',
#                  'unemployed', 
#                  'employees_social_security_at_work',
#                  'employees_social_security_at_residence'
                ]

In [15]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model(all_wide_data, params)

1 True
Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................1 False
Fitting all models.................................................................................................................................................................................................................................................................................................................................................................................................................2 True
Fitting all models..........................................................................................................................

## VAR with clusters

In [2]:
def evaluate_var_model_with_clusters(all_data, cluster_col, params, cluster_type='ags2'):
    
    # Create a runname
    run_name = 'cluster_var_' + str(len(params['relevant_cols'])) + 'cols_' + str(params['lag_value']) + 'lag_diff_' + str(params['second_diff'])
    
    ''' Utility Functions '''
    
    # Invert the transformation to get the real forecast
    def invert_transformation(df_train, df_forecast, second_diff=False):
        
        """Revert back the differencing to get the forecast to original scale."""
        df_fc = df_forecast.copy()
        columns = df_train.columns
        for col in columns:        
            # Roll back 2nd Diff
            if second_diff:
                df_fc[str(col)+'_diff'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_diff'].cumsum()
            # Roll back 1st Diff
            df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_diff'].cumsum()
        return df_fc
    
    # Start the MLflow experiment 
    with mlflow.start_run(run_name=run_name):
        
        # Make a list of ags 2 
        cluster_list = list(all_wide_data[cluster_col].unique())
        
        # Establish total error metrics
        total_mse, total_rmse, total_mape = 0, 0, 0
        count = 1
        # Iterate through the clusters 
        for cluster in cluster_list:
            print("Cluster: ", cluster)
        
            
            # Get all kreis in the clusters
            cluster_data = all_wide_data[all_wide_data[cluster_col] == cluster]

            ''' Create a dataframe of the unemployment rate '''
            kreis_list = list(cluster_data['ags5'].unique())
            
            # Check if the kreis has a single item
            if len(kreis_list) == 1:
                print("Single item in the cluster. Hence skipping.")
                continue
            
            # Create the data 
            unemployment_rate_df = cluster_data[cluster_data['ags5']==kreis_list[0]][['date']].reset_index(drop=True) # start with the date column
            
            # Iterate through the kreis
            for kreis in kreis_list:
                filter_data = cluster_data[cluster_data['ags5'] == kreis].reset_index()
                col_name = kreis + '_unemployment_rate'
                unemployment_rate_df[col_name] = filter_data['unemployment_rate']

            # Convert date format 
            unemployment_rate_df['date'] = pd.to_datetime(unemployment_rate_df['date'])
            
            # Clip the data before 2010 and after 2019 
            start_date = '2010-01-01'
            end_date = '2020-01-01'
            mask = (unemployment_rate_df['date'] >= start_date) & (unemployment_rate_df['date'] < end_date)
            input_df = unemployment_rate_df[(mask)]

            input_df.set_index('date', inplace=True)
            
            ''' Modelling Section '''
            
            # Perform a train test split
            n_obs = params['num_predictions']
            df_train, df_test = input_df[0:-n_obs], input_df[-n_obs:]
            
            # Differentiate the data twice if asked else once
            if_second_diff = params['second_diff']

            df_differenced = df_train.diff().dropna()
            if if_second_diff:
                df_differenced = df_differenced.diff().dropna()
                
            # Get the lag value 
            lag_value = params['lag_value']

            # Feed the model data
            model = VAR(df_differenced)

            # Fit the model to lag
            model_fitted = model.fit(lag_value)
            
            '''Forecasting section'''
            # Get the lag order
            lag_order = model_fitted.k_ar

            # Input data for forecasting
            forecast_input = df_differenced.values[-lag_order:]

            # Get the forecasts            
            fc = model_fitted.forecast(y=forecast_input, steps=n_obs)
            df_forecast = pd.DataFrame(fc, index=input_df.index[-n_obs:], columns=input_df.columns + '_diff')

            # Invert the forecast results
            df_results = invert_transformation(df_train, df_forecast, second_diff=if_second_diff)
            final_results = df_results.filter(regex='_forecast')
            
            ''' Evaluation '''
            
            # Iterate through all the kreis in the results 
            for kreis in kreis_list:
                
                forecast = df_results[kreis + '_unemployment_rate_forecast'].values
                actual = df_test[kreis + '_unemployment_rate']
                                 
                mse = mean_squared_error(actual, forecast, squared=True)
                rmse = mean_squared_error(forecast, actual, squared=False)
                mape = np.mean(np.abs(forecast - actual)/np.abs(actual))

                # add to the totals
                total_mse += mse
                total_rmse += rmse
                total_mape += mape
                print("kreis evaluated:", count)
                count+=1

        ''' Log metrics '''
        count -= 1
        print(count)
        # Calculate errors
        average_mse = total_mse/count
        average_rmse = np.sqrt(average_mse)
        average_mape = total_mape/count

        # Log model 
#         mlflow.statsmodels.log_model(model, "var_model")

        # log params
        mlflow.log_params(params)
        mlflow.log_param("cluster_column", cluster_col)
        mlflow.log_param("cluster_type", cluster_type)
        
        # Log errors
        mlflow.log_metric("mse", average_mse)
        mlflow.log_metric("rmse", average_rmse)
        mlflow.log_metric("mape", average_mape)


In [3]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
                 'registerd_jobs',
                 'unemployed', 
                 'employees_social_security_at_work',
                 'employees_social_security_at_residence'
                ]

In [4]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=all_wide_data, params=params, cluster_type='bundesland', cluster_col='ags2')

1 True


NameError: name 'all_wide_data' is not defined

### Clusters using PCA

In [72]:
relevant_cols = ['number_of_company_deletions', 
                 'number_of_company_liquidations',
                 'number_of_start_ups',
                 'unemployment_rate',
                 'registerd_jobs',
                 'unemployed', 
                 'employees_social_security_at_work',
                 'employees_social_security_at_residence'
                ]

In [64]:
# read the pca clusters by Amit 
cluster1 = pd.read_csv('df_final_stationary.csv', converters={'ags5':str, 'cluster':str})
cluster1_input = add_column_to_main_data(all_wide_data, cluster1, 'cluster')

original cluster data
0    332
2     67
1      2
Name: cluster, dtype: int64
New data
0    332
2     67
1      2
Name: cluster, dtype: int64


In [65]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster1_input, params=params, cluster_type='pca clusters with correct count', cluster_col='cluster')

1 True
Cluster:  0
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

Cluster:  0
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

2 True
Cluster:  0
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

kreis evaluated: 400
kreis evaluated: 401
401
2 False
Cluster:  0
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kre

3 True
Cluster:  0
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

...

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

### kmodes clusters

In [66]:
# Read the data 
cluster2 = pd.read_csv('kmodes3.csv', converters={'ags5':str, 'cluster':str})
print(cluster2.shape)
cluster2.head()

(401, 3)


,kreis,ags5,cluster
0,"Flensburg, Stadt",1001,1
1,"Kiel, Landeshauptstadt",1002,2
2,"Lübeck, Hansestadt",1003,2
3,"Neumünster, Stadt",1004,0
4,Dithmarschen,1051,1


In [67]:
cluster2_input = add_column_to_main_data(all_wide_data, cluster2, 'cluster')

original cluster data
2    190
0    114
1     97
Name: cluster, dtype: int64
New data
2    190
0    114
1     97
Name: cluster, dtype: int64


In [68]:
# intial params 
params = {
    'num_predictions' : 3,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster2_input, params=params, cluster_type='kmodes clusters', cluster_col='cluster')

1 True
Cluster:  1
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

Cluster:  1
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

...

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva

Cluster:  1
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

## tsne clusters


In [73]:
# Read the tsne data
cluster3 = pd.read_csv('df_tsne_cluster.csv', converters={'ags5':str, 'cluster':str})
print(cluster3.shape)
cluster3.head()

(401, 179)


,data_index,cluster,kreis,ags5,ags2,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,...,room_type_location,district_settlement_structure,type_of_settlement_structure,urban_/_rural,metropolitan_region,metropolitan_area,east_west,border_proximity,support_area_status,eligible_area
0,0,2,"Flensburg, Stadt",1001,1,92,500,35,240,2901,...,2,4,3,2,99,99,1,1,C,1
1,1,2,"Kiel, Landeshauptstadt",1002,1,92,460,37,268,2037,...,2,1,2,1,99,99,1,0,C/D,1
2,2,2,"Lübeck, Hansestadt",1003,1,90,532,37,297,1927,...,1,1,2,1,5,99,1,0,C/D,1
3,3,2,"Neumünster, Stadt",1004,1,85,588,37,316,1648,...,2,3,2,2,5,99,1,0,D,1
4,4,0,Dithmarschen,1051,1,51,1864,35,448,3517,...,4,4,3,2,5,99,1,0,C,1


In [74]:
cluster3_input = add_column_to_main_data(all_wide_data, cluster3, 'cluster')

original cluster data
0    179
1    129
2     93
Name: cluster, dtype: int64
New data
0    179
1    129
2     93
Name: cluster, dtype: int64


In [76]:
# intial params 
params = {
    'num_predictions' : 12,
    'lag_value': 4, 
    'relevant_cols': relevant_cols,
    'second_diff': True,
}

# Try a combination of lag and diff values 
lags = [i for i in range(1, 10)]
diff = [True, False]

for l in lags: 
    for d in diff:
        print(l, d)
        params['second_diff'] = d
        params['lag_value'] = l
    
        evaluate_var_model_with_clusters(all_data=cluster3_input, params=params, cluster_type='tsne clusters', cluster_col='cluster')

1 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

1 False
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eva

kreis evaluated: 400
kreis evaluated: 401
401
2 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
krei

Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

kreis evaluated: 395
kreis evaluated: 396
kreis evaluated: 397
kreis evaluated: 398
kreis evaluated: 399
kreis evaluated: 400
kreis evaluated: 401
401
3 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42

...

Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 

8 False
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eva

9 True
Cluster:  2
kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis eval

kreis evaluated: 1
kreis evaluated: 2
kreis evaluated: 3
kreis evaluated: 4
kreis evaluated: 5
kreis evaluated: 6
kreis evaluated: 7
kreis evaluated: 8
kreis evaluated: 9
kreis evaluated: 10
kreis evaluated: 11
kreis evaluated: 12
kreis evaluated: 13
kreis evaluated: 14
kreis evaluated: 15
kreis evaluated: 16
kreis evaluated: 17
kreis evaluated: 18
kreis evaluated: 19
kreis evaluated: 20
kreis evaluated: 21
kreis evaluated: 22
kreis evaluated: 23
kreis evaluated: 24
kreis evaluated: 25
kreis evaluated: 26
kreis evaluated: 27
kreis evaluated: 28
kreis evaluated: 29
kreis evaluated: 30
kreis evaluated: 31
kreis evaluated: 32
kreis evaluated: 33
kreis evaluated: 34
kreis evaluated: 35
kreis evaluated: 36
kreis evaluated: 37
kreis evaluated: 38
kreis evaluated: 39
kreis evaluated: 40
kreis evaluated: 41
kreis evaluated: 42
kreis evaluated: 43
kreis evaluated: 44
kreis evaluated: 45
kreis evaluated: 46
kreis evaluated: 47
kreis evaluated: 48
kreis evaluated: 49
kreis evaluated: 50
kreis eva